In [13]:
import os
import pandas as pd
import sqlite3


In [14]:
#region VERİTABANI OLUŞTURMA VE VERİ AKTARMA
current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'worldcities.csv')
df = pd.read_csv(#dataframe'i oku
    file_path
)
#sayılsal kolonları dönüştür 
df["population"] =pd.to_numeric(df["population"], errors='coerce')
#sqlite veritabanına bağlan
connection = sqlite3.connect("worldcities.db")
#dataframii tabloya aktar
df.to_sql("cities", connection, if_exists="replace", index=False)
#endregion  

48059

In [15]:
queries = {
    "en_kalabalik_10":
    """
        SELECT city, country, population 
        FROM cities 
        ORDER BY population DESC 
        LIMIT 10;
    """,
    "turkiye_sehirleri":
    """
        SELECT city, population 
        FROM cities 
        WHERE country = 'Turkey' 
        ORDER BY population DESC
        limit 10;
    """,
    "ulke_bazli_nufus":
    """
        SELECT country, SUM(population) as total_pop
        FROM cities 
        GROUP BY country 
        ORDER BY total_pop DESC 
        LIMIT 5;
    """
}

seçilen sorguyu çalıştır

In [16]:
selected_query = "en_kalabalik_10"#örnek olarak bir sorgu seç
print(pd.read_sql_query(queries[selected_query],connection))

          city       country  population
0        Tokyo         Japan  37785000.0
1      Jakarta     Indonesia  33756000.0
2        Delhi         India  32226000.0
3    Guangzhou         China  26940000.0
4       Mumbai         India  24973000.0
5       Manila   Philippines  24922000.0
6     Shanghai         China  24073000.0
7    São Paulo        Brazil  23086000.0
8        Seoul  Korea, South  23016000.0
9  Mexico City        Mexico  21804000.0


her sorguyu çalıştır ve sonucu yazdır

In [17]:
for item in queries:
    print(f"\nSorgu: {item}")
    print(pd.read_sql_query(queries[item], connection))
print("Sorgular tamamlandı.")


Sorgu: en_kalabalik_10
          city       country  population
0        Tokyo         Japan  37785000.0
1      Jakarta     Indonesia  33756000.0
2        Delhi         India  32226000.0
3    Guangzhou         China  26940000.0
4       Mumbai         India  24973000.0
5       Manila   Philippines  24922000.0
6     Shanghai         China  24073000.0
7    São Paulo        Brazil  23086000.0
8        Seoul  Korea, South  23016000.0
9  Mexico City        Mexico  21804000.0

Sorgu: turkiye_sehirleri
         city  population
0    Istanbul  14441000.0
1      Ankara   5864049.0
2       Bursa   3101833.0
3       İzmir   2965900.0
4       Konya   2320241.0
5   Gaziantep   2130432.0
6  Diyarbakır   1791373.0
7       Adana   1765981.0
8     Kayseri   1434357.0
9      Samsun   1356079.0

Sorgu: ulke_bazli_nufus
         country     total_pop
0          China  1.360846e+09
1          India  5.146940e+08
2  United States  3.794267e+08
3         Brazil  1.911845e+08
4          Japan  1.848941e+08
So

SORGULARI YAZABİLECEĞİMİZ BİR SORGU FONKSİYONU YAZALIM


In [18]:
# yardımcı fonksiyon queryleri alıp çalıştırır
def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None

In [19]:
#popülasyonu 1 milyondan fazla olan şehirleri al ilk 5 tanesini göster
MY_QUERY="""
select city , country
from cities
where population > 1000000
limit 5;
"""
df_query_result = run_query(MY_QUERY)

if df_query_result is not None:
    print(df_query_result)

        city    country
0      Tokyo      Japan
1    Jakarta  Indonesia
2      Delhi      India
3  Guangzhou      China
4     Mumbai      India


In [ ]:
run_query(""""
SELECT city, country, population
          from cities
          where country = 'Turkey' AND population > 500000
          order by population DESC
          limit 6;
"""
)

        city      country  population
0      Tokyo        Japan  37785000.0
1    Jakarta    Indonesia  33756000.0
2      Delhi        India  32226000.0
3  Guangzhou        China  26940000.0
4     Mumbai        India  24973000.0
5     Manila  Philippines  24922000.0


In [21]:
#kanadanın en kalabalık 5 şehri
print(pd.read_sql_query(
    "select city from cities "
    "where country='Canada' "
    "order by population desc limit 5",
    connection
))

        city
0    Toronto
1   Montréal
2  Vancouver
3    Calgary
4   Edmonton


In [22]:
#almanyanın en kalabalık şehri
print(pd.read_sql_query(
    "select city , population " 
    "from cities " 
    "where country ='Germany' " 
    "and population=(select max(population) from cities where country='Germany')",
    connection
))

     city  population
0  Berlin   4679500.0


bağlantıyı kapat

In [23]:
connection.close()